In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda, Dropout, BatchNormalization, Add, GlobalAveragePooling2D, concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

2024-06-29 00:42:54.119046: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-29 00:42:54.377908: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-29 00:42:55.774494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

# Function to load images from a folder
def load_images_from_folder(folder, augmentation=False):
    images = []
    datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                                 shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest') if augmentation else None

    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = preprocess_image(img_path)
        if img is not None:
            if augmentation:
                img = np.expand_dims(img, axis=0)
                for batch in datagen.flow(img, batch_size=1):
                    images.append(batch[0])
                    break
            else:
                images.append(img)

    return np.array(images)


In [3]:

def preprocess_image(image_path):
    # Load the image in grayscale
    img = load_img(image_path, color_mode='grayscale', target_size=(128, 128))
    img = img_to_array(img).astype('uint8')
    
    # Apply edge detection
    edges = cv2.Canny(img, threshold1=30, threshold2=100)
    
    # Dilate the edges to broaden the lines
    kernel = np.ones((3,3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    
    # Invert the edges: detected edges should be black, and the rest should be white
    edges = cv2.bitwise_not(edges)
    
    # Normalize the image
    edges = edges / 255.0
    edges = np.expand_dims(edges, axis=-1)
    
    return edges


In [4]:

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if img_path is not None:
            processed_img = preprocess_image(img_path)
            images.append(processed_img)
    return images


In [5]:

def create_dataset(base_path):
    X = []
    y = []
    for person_id in range(1, 65):  # Assuming you have 64 persons
        if person_id in [5,7,8,10,11]:
            continue
        person_id = str(person_id).zfill(3)
        real_folder = os.path.join(base_path, person_id)
        forge_folder = os.path.join(base_path, f"{person_id}_forg")

        real_images = load_images_from_folder(real_folder)
        forge_images = load_images_from_folder(forge_folder)

        for img in real_images:
            X.append(img)
            y.append(0)  # Label for genuine signatures

        for img in forge_images:
            X.append(img)
            y.append(1)  # Label for forged signatures

    X = np.array(X)
    y = np.array(y)
    X = X / 255.0  # Normalize the images
    X = X.reshape(-1, 128, 128, 1)  # Reshape for the CNN
    return X, y

base_path = 'sign_data/train'
X, y = create_dataset(base_path)

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
import cv2
# Display a single image before and after preprocessing
def display_image_before_after(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(img, (128, 128))
    processed_img = preprocess_image(file_path)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(img_resized, cmap='gray')
    plt.subplot(1, 2, 2)
    plt.title('Processed Image')
    plt.imshow(processed_img, cmap='gray')
    plt.show()

# Example image display
example_image_path = 'real.jpeg'  # Provide a valid image path here
display_image_before_after(example_image_path)


NameError: name 'preprocess_image' is not defined

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def build_complex_model():
    model = Sequential()
    
    # First convolutional block
    model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Second convolutional block
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Third convolutional block
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Fourth convolutional block
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    
    # Flatten and dense layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_complex_model()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Save the model
model.save('complex_signature_verification_model123.keras')


/home/ramachandra/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-29 00:43:01.623568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-29 00:43:01.715727: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download 

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 712ms/step - accuracy: 0.5470 - loss: 1.8664 - val_accuracy: 0.5081 - val_loss: 0.7144
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 22s 716ms/step - accuracy: 0.6453 - loss: 1.0076 - val_accuracy: 0.5081 - val_loss: 0.9483
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 655ms/step - accuracy: 0.6337 - loss: 0.9425 - val_accuracy: 0.5081 - val_loss: 1.3501
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 651ms/step - accuracy: 0.6745 - loss: 0.7912 - val_accuracy: 0.5081 - val_loss: 1.6758
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 653ms/step - accuracy: 0.7270 - loss: 0.6510 - val_accuracy: 0.5081 - val_loss: 2.1206
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 651ms/step - accuracy: 0.7753 - loss: 0.4885 - val_accuracy: 0.5081 - val_loss: 2.6166
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 652ms/step - accuracy: 0.8087 - loss: 0.4329 - val_accuracy: 0.5081 - val_loss: 2.9597
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 653ms/step - accuracy: 0.8175 - loss: 0.4053 - val_accu

In [10]:
def preprocess_image1(image_path):
    processed_img = preprocess_image(image_path)
    processed_img = processed_img / 255.0
    processed_img = processed_img.reshape(1, 128, 128, 1)
    return processed_img

def predict_signature(model, real_signature_path, test_signature_path):
    real_img = preprocess_image1(real_signature_path)
    test_img = preprocess_image1(test_signature_path)
    
    real_pred = model.predict(real_img)
    test_pred = model.predict(test_img)
    
    difference = np.abs(real_pred - test_pred)
    return difference

# Example usage
real_signature_path = 'img1.jpeg'  # Provide a valid image path
test_signature_path = 'img1.jpeg'  # Provide a valid image path
difference = predict_signature(model, real_signature_path, test_signature_path)
print(f"Difference: {difference}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Difference: [[0.]]
